# PI Tag Pulls - UNDER DEVELOPMENT
This workbook is used to pull data and create an output table for the desired tags in either a csv or excel format.  Reason for the program:
* Process engineers prefer Excel Workbooks,
* Excel connection to PI is slow,
* Excel crashes with large data pulls...and still needing data granularity.
* Current MWR process use Alteryx with an output that is slow and the output is not user friendly.

 Process
 1. The desired PI tags are first listed in an Excel workbook,
 2. The program will extract the PI tags and use the PiConnect to connect to PI AF,
 3. Data is extracted and placed into a flat file format
 4. Data is output into another excel workbook/worksheet or csv file.  

 Potential additions to code: 
 * Data from LIMS system is sent to PI.  Need to check if data points can be extracted without a interpolated value typically used in PI.  MWR currently extracts lab data using PowerBI. This is a visualization package and not a data manipulation package.  This could be developed as a separate file.
 * Data Type Column to select from PI or Lab data that is optional to send to the final report.
 * Units column that is optional to send to the final report

Created by: TW  
Created on: 2025-05-07  
Environment: python_20240807

## Libraries

In [33]:
#connect to PI Server
import  PIconnect as PI
from  PIconnect.PIConsts import SummaryType

#manage time
import pytz
from datetime import datetime

#standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#other libraries
import re
import pathlib
import xlsxwriter
import openpyxl
import os


## Definitions

#### Extract PI Tags from Excel
Convert an Excel spreadsheet that has the tag name and PI tag in to a dict. Note that process engineers love their lists of stuff in Excel spreadsheets.  Not efficient but that is easiest solution for them.

In [22]:
### Add logging and unit tests
###Add output of removed records
def fn_PI_tags_from_excel(file_name: str ):
    """This function pulls PI_tags that are listed in an Excel Spreadsheet into a dataframe.  Excel spreadsheet should have 2 columns with the first column as the "Tag Name" and second column as the "Tag".

    Args:
        file_name (str): the name of hte excel workbook with the PI tags listed.  If it is not in the same directory, then the this is the file path for the Excel workbook.
    """
    # Check for excel format
    try:
        df_PI_tags = pd.read_excel(file_name, sheet_name="data_dictionary")
    except:
        raise TypeError("fn_PI_tags_from_excel: This must be an Excel file.  Check file name and path.")

    #clean PI tags
    df_PI_tags_cln = df_PI_tags.dropna(axis=0, how="any", subset=['Tag Name', 'Tag', 'Summary Type'])
    df_PI_tags_cln = df_PI_tags_cln.drop_duplicates(keep="first")

    #create a df of removed records
    df_PI_tags_removed = df_PI_tags.merge(df_PI_tags_cln, on=['Tag Name', 'Tag', 'Summary Type'], how="outer", indicator= True)

    df_PI_tags_removed = df_PI_tags_removed[df_PI_tags_removed['_merge'] != 'both']

    df_PI_tags_removed = df_PI_tags_removed.drop(columns=['_merge'], axis=1)

    #Reset Indexes
    df_PI_tags_cln = df_PI_tags_cln.reset_index(drop=1)
    df_PI_tags_removed = df_PI_tags_removed.reset_index(drop=1)

    return df_PI_tags_cln, df_PI_tags_removed



In [ ]:
fn_PI_tags_from_excel(file_name='PI_tag_list.xlsx')[0]

,Tag Name,Tag,Summary Type,Data Type_x,Data Type_y


#### Generate dataframe with requested data

In [ ]:
###  Needs logging and unit testing
###### add sum_type in second row or onto tag name
######## verify function  types

def fn_create_results_dataframe(df: pd.DataFrame,
                           start_time: str | datetime = "-30d",
                           end_time: str | datetime = "00:00:00",
                           interval: str = "15m"):
                           #sum_type: str = 'AVERAGE' #sum type from spreadsheet instead of input

    """This function takes in a dataframe that contains a column with a 'tag name' and a second column with the 'tag'.  It creates a new dataframe that where the tag name becomes the name of the columns.  The tag is the path to the PI attribute which is used to look up the requested data.

    Args:
        df (pd.DataFrame): a dataframe that contain the 'tag name' in the first column and the (PI) 'tag' in the second column.

        start_time ([str, datetime], optional): The start time for the data collection. Uses PI System Time abbreviations. Defaults to "-30d".

        end_time ([str, datetime], optional): The end time for the data collection. Uses PI System Time abbreviations. Defaults to "00:00".

        interval (str, optional): The  time interval between each record. Uses PI System Time abbreviations. Defaults to "15m".

        sum_type (str, optional): The PI SummaryTypes. Some common values are: "AVERAGE", "COUNT", "MAXIMUM", "MINIMUM", "RANGE", "STD_DEV", and "TOTAL". Defaults to 'AVERAGE'.

    Returns:
        pd.DataFrame: A dataframe that contains columns of data.  The column headers are the tag names provided and the data in each column is based upon the function parameter request.

    Notes:
        index is listed as local Denver timezone
    """

    # # verify parameters

    #### HASHED Sum_type as it is taken from  Excel Spreadsheet
    # lst_sum_type = ["ALL", "ALL_FOR_NON_NUMERIC", "AVERAGE", "COUNT", "MAXIMUM", "MINIMUM", 'NONE', "PERCENT_GOOD", "POP_STD_DEV", "RANGE", "STD_DEV", "TOTAL", "TOTAL_WITH_UOM"]

    # try:
    #     lst_sum_type.index(sum_type)
    # except:
    #     raise TypeError("SummaryType is not from provided list.")

    #create result dataframe
    df_results = pd.DataFrame()

    #iterate through the tags
    for i in df.index:
        df_labels = pd.DataFrame()
        
        lbl = df.iloc[i,0] #tag names to pd column titles
        tag = str(df.iloc[i,1]) # make ta a string for python readability
        tag_split = tag.rsplit(sep="|") #split the PI element and attribute
        sum_type = df.iloc[i,2]
        #collect the data from PI and adds to results dataframe
        tag_data = db.descendant(tag_split[-2]).attributes[tag_split[-1]].summaries(start_time, end_time, interval, SummaryType.AVERAGE)

        df_results[lbl] = tag_data

    #Adjust to Denver timezone
    df_results = df_results.tz_convert('America/Denver')
    df_results.index = df_results.index.strftime('%Y-%m-%d %H:%M:%S')

    return df_results




In [70]:
df1 = pd.DataFrame({"Name": [1,2,3]})
new_columns = pd.MultiIndex.from_tuples([
    ('Name', 'subheader1'),  # Replace with your desired headers
], names=['Level1', 'Level2'])
df1.columns = new_columns
df1

Level1,Name
Level2,subheader1
0,1
1,2
2,3


#### Create Excel or CSV Output

In [ ]:
#### Needs logging and unit testing

##### "new" should not overwrite existing file - create numbering
##### selection of different sheet name
def fn_create_output_table(df: pd.DataFrame, file_name: str, file_output: str = 'existing',**kw):
    """This function takes in a dataframe and creates an output file.  The output file can be a csv or an excel workbook.

    Args:
        df (pd.DataFrame): A dataframe that contains the data to be output.

        file_name (str): The name of the output file.  If it is not in the same directory, then the this is the file path for the output file.  Files should have a file extension for a csv or excel file (.csv, .xlsx, .xls)

        file_output(str): Tells function to either add the data to a new excel file or append the data to the existing excel file. Options are "new" or "existing". Default is "existing".

        if_sheet_exists (str, optional): Used only with existing excel files.  Determines what to do with data.  Select from "error", "new", "replace", "overlay".  See pandas ExcelWriter api for description. Default is "replace".

    Returns:
        This function outputs results to a csv or excel file.
    """
    #validate filename type
    if not isinstance(file_name, str):
        raise TypeError('fn_create_output_table: file_name needs to be a string')

    # default sheet_name
    if not 'sheet_name' in kw.keys():
        kw['sheet_name'] = "PI Tag Output"

    #file_output validation and ExcelWriter adjustments
    if file_output == "existing":
        #default if_sheet_exists value
        if not "if_sheet_exists" in kw.keys():
            kw['if_sheet_exists'] = "replace"

        #validate if_sheet_exists
        if not kw['if_sheet_exists'] in ['error', 'new', 'replace', 'overlay']:
            raise ValueError('fn_create_output_table: if_sheet_exists needs to be: error, new, replace, or overlay.')

        md = "a" #mode for appending sheet

    elif file_output == "new":
        md = "w+" #mode for read/write

        kw['if_sheet_exists'] = None #no if_sheet_exists for creating new file

        #stop overwriting of an existing file
        if os.path.isfile(file_name):
            raise NameError(f"fn_create_output_table: {file_name} already exists. Use different file name or change file_output to 'existing'.")
    else:
        raise TypeError('fn_create_output_table: output_type is either exist or new.')

    #extract file extension from file_name
    file_ext = file_name.rsplit(sep=".")[-1].lower()

    #validate file extension as excel or csv, and generate files
    if file_ext == "csv":
        df.to_csv(file_name, index=True)
        print('csv works')
    elif file_ext in ['xlsx', 'xls']:
        with pd.ExcelWriter(file_name,
                            engine='openpyxl',
                            mode= md,
                            if_sheet_exists=kw['if_sheet_exists']
                            ) as writer:
            df.to_excel(writer, sheet_name=kw['sheet_name'], index=True)
    else:
        raise TypeError('fn_create_output_table: file_name needs to have an csv or excel file extension type (.csv, .xlsx, .xls).')

In [72]:

fn_create_output_table(df=df1, file_name='test.xlsx', file_output='existing', sheet_name="pig", if_sheet_exists="new")

## Main function

In [51]:
#Needs to be functionalized,
#needs output to excel or a csv.
#needs unit testing and logging.

#inputs
filename = "PI_tag_list.xlsx"


# make connection to the PI database
with PI.PIAFDatabase() as db:
    print(f"Connected to {db.server_name}")

#crate data frame of PI_tags
df_PI_tags = fn_PI_tags_from_excel(file_name=filename)
df_PI_tags_removed = df_PI_tags[1]
df_PI_tags_cln = df_PI_tags[0]

# PI request and result table creation
df_results = fn_create_results_dataframe(df=df_PI_tags_cln)

#save results to excel/csv
fn_create_output_table(df=df_results, file_name=filename, file_output = 'existing')





Connected to APPLEPI_AF
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [29]:
df_results


,S Platte Temp,SHT1 flow
2025-04-09 00:00:00,15.676,-1.825875
2025-04-09 00:15:00,15.676,-83.227764
2025-04-09 00:30:00,15.676,20.214584
2025-04-09 00:45:00,15.676,-24.640912
2025-04-09 01:00:00,15.676,-8.354832
...,...,...
2025-05-08 22:45:00,15.676,-0.206695
2025-05-08 23:00:00,15.676,-0.482275
2025-05-08 23:15:00,15.676,-0.256461
2025-05-08 23:30:00,15.676,0.000000
